## Read Data From CSV

In [1]:
import csv
from unidecode import unidecode
latitude = []
longitude = []
city_name = []
county_name = []
with open('Datasets/ro.csv', newline='') as csvfile:
    DictReader = csv.DictReader(csvfile)
    ### Get latitude and longitude from csv
    for row in DictReader:
        city = unidecode(row['city'])
        city = city.replace("AY", 's')
        city = city.replace("APS", 't')
        city = city.replace("Af", 'a')
        city = city.replace("AC/", 'a')
        city_name = city_name + [city]
        county = unidecode(row['admin_name'])
        county = county.replace("AY", 's')
        county = county.replace("APS", 't')
        county = county.replace("Af", 'a')
        county = county.replace("AC/", 'a')
        county_name = county_name + [county]
        latitude = latitude + [row['lat']]
        longitude = longitude + [row['lng']]

## Generate the graphs

In [2]:
import networkx as nx
import geopandas as gpd
import osmnx as ox

G = []
hospitals = []
saved_city = []
for i in range(1, len(city_name)):
    try:
        location = city_name[i] + ", " + county_name[i]
        
        x = ox.graph_from_place(location, network_type="drive")
        if x:
            saved_city.append(location)
            G.append(x)
            hospitals.append(ox.features.features_from_place(location, tags={"amenity": "hospital", "amenity":"clinic", "healthcare":"hospital", "emergency":"yes"}))

    except Exception as e:
        print(f"Error processing {city_name[i]}: {e}")

Error processing Roman: No data elements in server response. Check log and query location/tags.
Error processing Voluntari: Found no graph nodes within the requested polygon
Error processing Medgidia: No data elements in server response. Check log and query location/tags.


# GRAPH MEASUREMENTS

## Degree

Degree -- Definition
Number of links connected to the node ki
.

Average node degree -- Definition
Sum of the degrees and divided by the number of nodes.

In [3]:
def average_degree(G: nx.Graph) -> float:
    return sum(G.degree(n) for n in G.nodes) / len(G.nodes)

In [4]:
for i in range(0, len(G)):
    print(f"Average degree for {saved_city[i]} {average_degree(G[i])}")

Average degree for Cluj-Napoca, Cluj 4.511713933415536
Average degree for Iasi, Iasi 4.750678557580458
Average degree for Constanta, Constanta 4.7278708133971294
Average degree for Timisoara, Timis 4.646375902102291
Average degree for Brasov, Brasov 4.500451671183378
Average degree for Craiova, Dolj 4.653881278538813
Average degree for Galati, Galati 4.982316029663434
Average degree for Oradea, Bihor 4.634392334846193
Average degree for Ploiesti, Prahova 4.850704225352112
Average degree for Braila, Braila 5.711436950146628
Average degree for Arad, Arad 5.012997903563941
Average degree for Pitesti, Arges 4.610193826274228
Average degree for Bacau, Bacau 4.780707395498392
Average degree for Sibiu, Sibiu 4.659420289855072
Average degree for Targu-Mures, Mures 4.350640593391773
Average degree for Baia Mare, Maramures 4.293546544831525
Average degree for Buzau, Buzau 4.774381368267831
Average degree for Ramnicu Valcea, Valcea 4.3890577507598785
Average degree for Satu Mare, Satu Mare 4.5998

## Degree Hospital Nodes

In [5]:
# Open the file in write mode
with open(".\Datasets\GraphMeasurements\hospital_degrees.txt", "w") as file:
    for i in range(0, len(G)):
        try:
            # Extract hospital nodes
            hospital_nodes = []
            for _, hospital in hospitals[i].iterrows():
                point = hospital.geometry.centroid
                nearest_node = ox.distance.nearest_nodes(G[i], point.x, point.y)
                hospital_nodes.append(nearest_node)
            hospital_nodes = list(set(hospital_nodes))

            # Ensure there are hospital nodes
            if not hospital_nodes:
                file.write(f"No hospital nodes found for {saved_city[i]}\n")
            else:
                # Calculate the degree for each hospital node
                hospital_degrees = [G[i].degree(node) for node in hospital_nodes]
                avg_degree = sum(hospital_degrees) / len(hospital_degrees) if hospital_degrees else 0

                # Write the average degree to the text file
                file.write(f"Average degree of hospital nodes for {saved_city[i]}: {avg_degree:.4f}\n")

        except Exception as e:
            file.write(f"Error processing {saved_city[i]}: {e}\n")

<>:2: DeprecationWarning: invalid escape sequence '\D'
<>:2: DeprecationWarning: invalid escape sequence '\D'
C:\Users\Adrian\AppData\Local\Temp\ipykernel_10508\1539449993.py:2: DeprecationWarning: invalid escape sequence '\D'
  with open(".\Datasets\GraphMeasurements\hospital_degrees.txt", "w") as file:


## Hospital Clustering Coefficients
## What fraction of hospital neighbors are connected

In [6]:
def clustering_coef(G: nx.Graph, u: int):
    """
    Return the clustering coefficient of node `u` from graph `G`

    Parameters:
    -----------
    G: nx.Graph
        Input graph
    u: int
        Node to calculate the clustering coefficient of.

    Returns:
    --------
    clustering coeff: int

    """
    ei = 0
    for i in G.neighbors(u):  # For each neighbour `i` of `u`
        for j in G.neighbors(u):  # For each neighbour `j` of `u`
            if i != j:  # no self loops
                ei += G.has_edge(i, j)  # Check if it has edge and add it to the total edges
    return ei / (G.degree(u) * (G.degree(u) - 1))  # divide by 2 because we count the edges twice

In [7]:
with open(".\Datasets\GraphMeasurements\hospital_clustering_coefficients.txt", "w") as file:   
    for i in range(0, len(G)):
        try:
            hospital_nodes = []
            for _, hospital in hospitals[i].iterrows():
                point = hospital.geometry.centroid
                nearest_node = ox.distance.nearest_nodes(G[i], point.x, point.y)
                hospital_nodes.append(nearest_node)
            hospital_nodes = list(set(hospital_nodes))

            # Ensure there are hospital nodes
            if not hospital_nodes:
                file.write(f"No hospital nodes found for {saved_city[i]}\n")
            else:
                # Calculate the clustering coefficients for the hospital nodes
                hospital_clustering_coeffs = {node: clustering_coef(G[i], node) for node in hospital_nodes if G[i].degree(node) > 1}
                file.write(f"Clustering coefficients for {saved_city[i]}\n")
                # Print the clustering coefficients for the hospital nodes
                for hospital_node, clustering_coeff in hospital_clustering_coeffs.items():
                    file.write(f"Hospital node {hospital_node} has clustering coefficient {clustering_coeff:.4f}\n")
                file.write("\n")  # Add a newline for separation between cities

        except Exception as e:
            file.write(f"Error processing {saved_city[i]}: {e}\n")

<>:1: DeprecationWarning: invalid escape sequence '\D'
<>:1: DeprecationWarning: invalid escape sequence '\D'
C:\Users\Adrian\AppData\Local\Temp\ipykernel_10508\3932586345.py:1: DeprecationWarning: invalid escape sequence '\D'
  with open(".\Datasets\GraphMeasurements\hospital_clustering_coefficients.txt", "w") as file:
